In [ ]:
!pip install flask-ngrok

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import requests
from datetime import datetime

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/convert-date', methods=['GET'])
def convert_date():
    gregorian_date = request.args.get('date')
    after_sunset = request.args.get('after_sunset', 'false').lower() == 'true'

    if not gregorian_date:
        return jsonify({"error": "Missing date parameter. Use the format YYYY-MM-DD"}), 400

    api_url = f"https://www.hebcal.com/converter?cfg=json&date={gregorian_date}&g2h=1&strict=1"
    if after_sunset:
        api_url += "&gs=on"

    response = requests.get(api_url)

    if response.status_code != 200:
        return jsonify({"error": "Failed to fetch Hebrew date. Check the provided date."}), 400

    data = response.json()
    if "hebrew" not in data:
        return jsonify({"error": "Invalid response from the API"}), 500

    return jsonify({
        "gregorian_date": gregorian_date,
        "hebrew_date": data["hebrew"],
        "after_sunset": after_sunset
    })

if __name__ == '__main__':
    while True:
        print("\nWelcome to the Date Converter! Please enter a Gregorian date")
        gregorian_date = input("Gregorian date in the format YYYY-MM-DD (or type 'exit' to quit): ").strip()

        if gregorian_date.lower() == 'exit':
            print("Thank you for using the Date Converter! Goodbye!")
            break

        try:
            datetime.strptime(gregorian_date, "%Y-%m-%d")
        except ValueError:
            print("Invalid date. Please enter a valid Gregorian date in the format YYYY-MM-DD.")
            continue


        after_sunset = input("Is it after sunset? Please type 'yes' or 'no': ").strip().lower()
        if after_sunset not in ['yes', 'no']:
            print("Invalid input for after sunset. Please enter 'yes' or 'no'.")
            continue


        after_sunset = after_sunset == 'yes'


        api_url = f"https://www.hebcal.com/converter?cfg=json&date={gregorian_date}&g2h=1&strict=1"
        if after_sunset:
            api_url += "&gs=on"

        response = requests.get(api_url)


        if response.status_code == 200:
            data = response.json()
            if "hebrew" in data:
                print(f"\nGregorian date: {gregorian_date}")
                print(f"Hebrew date: {data['hebrew']}")
            else:
                print("Error in converting the date.")
        else:
            print("Error in fetching the data. Please check the date you entered.")



Welcome to the Date Converter! Please enter a Gregorian date

Gregorian date: 2020-12-12
Hebrew date: כ״ו בְּכִסְלֵו תשפ״א

Welcome to the Date Converter! Please enter a Gregorian date

Gregorian date: 2020-12-12
Hebrew date: כ״ו בְּכִסְלֵו תשפ״א

Welcome to the Date Converter! Please enter a Gregorian date
